In [ ]:
import cv2
import pandas as pd
import numpy as np
from deepface import DeepFace
import matplotlib.pyplot as plt

def is_black_frame(frame, threshold=15):
    """Check if frame is mostly black"""
    mean_brightness = np.mean(frame)
    return mean_brightness < threshold

def has_valid_face(frame, min_face_size=50):
    """Check if frame has a detectable face of reasonable size"""
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(min_face_size, min_face_size))
        return len(faces) > 0
    except:
        return False

def get_neutral_aus(frame_count, fps):
    """Return neutral AU values for frames without faces"""
    return {
        'frame': frame_count,
        'timestamp': frame_count / fps,
        'AU01': 0.0,  # Inner Brow Raiser
        'AU02': 0.0,  # Outer Brow Raiser
        'AU04': 0.0,  # Brow Lowerer
        'AU05': 0.0,  # Upper Lid Raiser
        'AU06': 0.0,  # Cheek Raiser
        'AU07': 0.0,  # Lid Tightener
        'AU09': 0.0,  # Nose Wrinkler
        'AU10': 0.0,  # Upper Lip Raiser
        'AU12': 0.0,  # Lip Corner Puller (Smile)
        'AU14': 0.0,  # Dimpler
        'AU15': 0.0,  # Lip Corner Depressor
        'AU17': 0.0,  # Chin Raiser
        'AU20': 0.0,  # Lip Stretcher
        'AU23': 0.0,  # Lip Tightener
        'AU25': 0.0,  # Lips Part
        'AU26': 0.0,  # Jaw Drop
        'AU28': 0.0,  # Lip Suck
        'face_detected': False,
        'is_black_frame': True
    }

def emotions_to_aus(emotions):
    """Convert emotion probabilities to Action Unit intensities"""
    # Normalize emotions to 0-1 range
    happy = emotions.get('happy', 0) / 100
    sad = emotions.get('sad', 0) / 100
    angry = emotions.get('angry', 0) / 100
    surprise = emotions.get('surprise', 0) / 100
    fear = emotions.get('fear', 0) / 100
    disgust = emotions.get('disgust', 0) / 100
    
    aus = {}
    
    # Map emotions to Action Units based on FACS (Facial Action Coding System)
    aus['AU01'] = surprise * 0.8 + fear * 0.3  # Inner Brow Raiser
    aus['AU02'] = surprise * 0.9 + fear * 0.4  # Outer Brow Raiser
    aus['AU04'] = angry * 0.8 + disgust * 0.4  # Brow Lowerer
    aus['AU05'] = surprise * 0.7 + fear * 0.5  # Upper Lid Raiser
    aus['AU06'] = happy * 0.9  # Cheek Raiser (strong with happiness)
    aus['AU07'] = angry * 0.4 + disgust * 0.5  # Lid Tightener
    aus['AU09'] = disgust * 0.8  # Nose Wrinkler
    aus['AU10'] = disgust * 0.6 + angry * 0.3  # Upper Lip Raiser
    aus['AU12'] = happy * 1.0  # Lip Corner Puller (main smile AU)
    aus['AU14'] = happy * 0.6  # Dimpler
    aus['AU15'] = sad * 0.8  # Lip Corner Depressor
    aus['AU17'] = sad * 0.5 + disgust * 0.3  # Chin Raiser
    aus['AU20'] = fear * 0.6  # Lip Stretcher
    aus['AU23'] = angry * 0.5  # Lip Tightener
    aus['AU25'] = surprise * 0.8 + fear * 0.4  # Lips Part
    aus['AU26'] = surprise * 0.9 + fear * 0.5  # Jaw Drop
    aus['AU28'] = fear * 0.3  # Lip Suck
    
    for au in aus:
        aus[au] = min(1.0, max(0.0, aus[au]))
    
    return aus

# Upload video
# print("Upload your video:")
# uploaded = files.upload()
# video_path = list(uploaded.keys())[0]

# Process video
# cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture("/content/Daemahni_on_DaemahniGianna.mov")
results = []
frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)

print("Processing video - extracting Action Units only...")
print("Detecting and handling black frames...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_count % 30 == 0:
        
        if is_black_frame(frame):
            frame_data = get_neutral_aus(frame_count, fps)
            results.append(frame_data)
            
        elif not has_valid_face(frame):
            frame_data = get_neutral_aus(frame_count, fps)
            frame_data['is_black_frame'] = False
            results.append(frame_data)
            
        else:
            # Process with DeepFace
            try:
                analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                
                if isinstance(analysis, list):
                    analysis = analysis[0]
                
                # Convert emotions to AUs
                aus = emotions_to_aus(analysis['emotion'])
                
                frame_data = {
                    'frame': frame_count,
                    'timestamp': frame_count / fps,
                    'face_detected': True,
                    'is_black_frame': False
                }
                frame_data.update(aus)
                
                results.append(frame_data)
                
            except Exception as e:
                # If DeepFace fails, use neutral values
                frame_data = get_neutral_aus(frame_count, fps)
                frame_data['is_black_frame'] = False
                results.append(frame_data)
    
    frame_count += 1
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames...")

cap.release()

# Create results DataFrame
df = pd.DataFrame(results)
print(f"Successfully processed {len(df)} frames!")

# Show statistics
face_frames = df['face_detected'].sum()
black_frames = df['is_black_frame'].sum()
print(f"Frames with faces: {face_frames}")
print(f"Black frames: {black_frames}")
print(f"Total processed frames: {len(df)}")

# Get AU columns only
au_columns = [col for col in df.columns if col.startswith('AU')]
print(f"Extracted Action Units: {au_columns}")

# Show sample results (AUs only)
print("\nSample Action Unit results:")
sample_cols = ['frame', 'timestamp', 'face_detected'] + au_columns[:5]
print(df[sample_cols].head())

# Plot Action Units over time
plt.figure(figsize=(16, 12))

# Plot all AUs
n_aus = len(au_columns)
n_cols = 4
n_rows = (n_aus + n_cols - 1) // n_cols

for i, au in enumerate(au_columns):
    plt.subplot(n_rows, n_cols, i+1)
    
    # Plot all frames
    plt.plot(df['timestamp'], df[au], alpha=0.6, color='lightblue', label='All frames')
    
    # Highlight frames with detected faces
    face_data = df[df['face_detected'] == True]
    if len(face_data) > 0:
        plt.plot(face_data['timestamp'], face_data[au], 
                color='blue', linewidth=2, label='Face detected')
    
    # Mark black frames
    black_data = df[df['is_black_frame'] == True]
    if len(black_data) > 0:
        plt.scatter(black_data['timestamp'], black_data[au], 
                   c='red', s=15, alpha=0.8, label='Black/No face', marker='x')
    
    plt.title(f'{au} - Action Unit Intensity')
    plt.xlabel('Time (seconds)')
    plt.ylabel('AU Intensity (0-1)')
    plt.ylim(-0.05, 1.05)
    plt.grid(True, alpha=0.3)
    
    # Only show legend on first subplot to avoid clutter
    if i == 0:
        plt.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Summary statistics for AUs
print("\nAction Unit Summary Statistics:")
au_stats = df[au_columns].describe()
print(au_stats.round(3))

# Most active AUs
print("\nMost Active Action Units (by mean intensity):")
au_means = df[df['face_detected'] == True][au_columns].mean().sort_values(ascending=False)
for au, intensity in au_means.head(10).items():
    au_names = {
        'AU01': 'Inner Brow Raiser', 'AU02': 'Outer Brow Raiser', 'AU04': 'Brow Lowerer',
        'AU05': 'Upper Lid Raiser', 'AU06': 'Cheek Raiser', 'AU07': 'Lid Tightener',
        'AU09': 'Nose Wrinkler', 'AU10': 'Upper Lip Raiser', 'AU12': 'Lip Corner Puller',
        'AU14': 'Dimpler', 'AU15': 'Lip Corner Depressor', 'AU17': 'Chin Raiser',
        'AU20': 'Lip Stretcher', 'AU23': 'Lip Tightener', 'AU25': 'Lips Part',
        'AU26': 'Jaw Drop', 'AU28': 'Lip Suck'
    }
    description = au_names.get(au, 'Unknown')
    print(f"  {au}: {intensity:.3f} - {description}")

# Save results
print(f"\nSaving results...")

# Complete dataset with all frames
df.to_csv('action_units_complete.csv', index=False)
files.download('action_units_complete.csv')

# Clean dataset with only face-detected frames
face_only_df = df[df['face_detected'] == True].copy()
if len(face_only_df) > 0:
    face_only_df.to_csv('action_units_faces_only.csv', index=False)
    files.download('action_units_faces_only.csv')
    print(f"Clean dataset: {len(face_only_df)} frames with faces")

# AU-only dataset (no metadata columns)
au_only_df = df[['timestamp'] + au_columns].copy()
au_only_df.to_csv('action_units_data_only.csv', index=False)
files.download('action_units_data_only.csv')

print("Done! Three files downloaded:")
print("  1. action_units_complete.csv - All frames including black screens")
print("  2. action_units_faces_only.csv - Only frames with detected faces") 
print("  3. action_units_data_only.csv - Just timestamps and AU values")